In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
cmap = matplotlib.cm.get_cmap('Spectral')

import numpy as np

from sklearn.datasets import make_moons, make_hastie_10_2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

In [ ]:
## Funciones útiles

def print_results(clfrgr):
    print('#####')
    print('{}.'.format(type(clfrgr).__name__))
                       
    train_score = clfrgr.score(X_train, y_train)
    test_score = clfrgr.score(X_test, y_test)
    print('Train score: ', train_score)
    print('Test score: ', test_score)

def print_results_boost(clfrgr):
    print('#####')
    print('{} con {} estimadores de tipo {}, con max_depth={}.'.format(type(clfrgr).__name__,
                                                                       clfrgr.n_estimators, 
                                                                       type(clfrgr.base_estimator).__name__,
                                                                       clfrgr.base_estimator.max_depth))
    train_score = clfrgr.score(X_train, y_train)
    test_score = clfrgr.score(X_test, y_test)
    print('Train score: ', train_score)
    print('Test score: ', test_score)

    return train_score, test_score

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.5, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    ax = plt.gca()
    
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "o", color=cmap(0), alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "yo", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

## Bagging

In [ ]:
X, y = make_moons(2000, shuffle=True, noise=0.2, random_state=1234)
size_train = int(0.8 * len(X))
X_train, X_test = X[:size_train], X[size_train:]
y_train, y_test = y[:size_train], y[size_train:]

plt.scatter(*X_train.T, c=y_train, marker='.')
#plt.scatter(*X_test.T, c=y_test, marker='^')

In [ ]:
dt = DecisionTreeClassifier(max_depth=12, random_state=107)
dt = dt.fit(X_train, y_train)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='poly'))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

### Decision Trees

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(DecisionTreeClassifier(max_depth=12, random_state=107), n_estimators=500, n_jobs=-1, 
                        oob_score=True, max_samples=1000, bootstrap=True)
clf = clf.fit(X_train, y_train)

In [ ]:
print(clf.oob_decision_function_.shape)
print(clf.oob_score_)
print(clf.oob_decision_function_[:5])

In [ ]:
plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)

### Extra trees

In [ ]:
from sklearn.tree import ExtraTreeClassifier

dt = ExtraTreeClassifier(max_depth=12, random_state=107)
dt = dt.fit(X_train, y_train)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='poly'))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

In [ ]:
clf = BaggingClassifier(ExtraTreeClassifier(max_depth=12, random_state=107), n_estimators=500, n_jobs=-1, 
                        oob_score=False, max_samples=1000, bootstrap=True)
clf = clf.fit(X_train, y_train)

plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)

## Bias-Variance

(mostrado con *bagging*)

Podemos ver el efecto de usar modelos con más o menos flexibilidad a la hora de realizar comités. 

En este caso, arranquemos con un DT, con profundidad 1. Obviamente, no va a alcanzar para explicar este dataset.

In [ ]:
dt = DecisionTreeClassifier(max_depth=1, random_state=107)
dt = dt.fit(X_train, y_train)

# dt = ExtraTreeClassifier(max_depth=7, random_state=107)

# svm_pipe = Pipeline(steps=[('scaler', StandardScaler()), 
#                            ('svc', SVC(kernel='rbf', gamma=1.0, C=0.6))])
# svm_pipe = svm_pipe.fit(X_train, y_train)

plot_decision_boundary(dt, X_train, y_train, contour=True, alpha=0.1)
print_results(dt)

Pero, ¿qué pasará si combinamos 500 de estos? ¿Lo lograremos?

In [ ]:
clf = BaggingClassifier(dt, n_estimators=2000, n_jobs=-1, 
                        oob_score=False, max_samples=0.2, bootstrap=True)
clf = clf.fit(X_train, y_train)

plot_decision_boundary(clf, X_train, y_train, alpha=0.1)
print_results(clf)

No no no!

Volvamos arriba y veamos qué nivel de flexibilidad le tenemos que dar al DT inicial para que el ensemble funcione bien.

***
**Ejercicio**: probemos después otros algoritmos de base, como el `ExtraTreeClassifier`. ¿Qué ventaja hay?

## Boosting

Para mostrar el poder de boosting, hagamos un nuevo dataset.

In [ ]:
from sklearn.datasets import make_hastie_10_2

X, t = make_hastie_10_2(random_state=0)

print(X.shape)

X_train, X_val = X[2000:], X[:2000]
t_train, t_val = t[2000:], t[:2000]

Este dataset consiste en diez features normales estándares, con los siguientes labels: 

`t[i] = 1 if np.sum(X[i] ** 2) > 9.34 else -1`

Es decir, es una hiperesfera adentro de una hiperesfera hueca.

In [ ]:
import pandas as pd
import seaborn as sns

dd = pd.DataFrame.from_records(X_train)
dd['target'] = t_train

#ss.pairplot(dd.iloc[::20, :5], hue='target')
#ss.pairplot(dd.iloc[::20], hue='target', vars=range(5), markers=',')
sns.pairplot(dd.iloc[::20], hue='target', vars=range(4), markers='.')

### Adaptive Boosting (AdaBoost)

Veamos como funciona `AdaBoost` en un caso así. Combinemos DTs con profundidad máxima 2.

In [ ]:
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=1.0)

clf.fit(X_train, t_train)
print_results_boost(clf)

Obviamente esto es mucho mejor que un predictor solo, que es casi lo mismo que nada.

In [ ]:
dt = DecisionTreeClassifier(max_depth=1)
dt.fit(X_train, t_train)
print_results(dt)

In [ ]:
for nesti in [10, 20, 50, 75]:
    clf.set_params(n_estimators=nesti)
    clf.fit(X_train, y_train)
    print_results_boost(clf)

### Early stopping

Pero la pregunta que se impone es cuándo me detengo. Una forma de evitar el sobreajuste es detenerse cuando el error en el conjunto de validación llegue a un mínimo. Esta estrategia de regularización se llama *early stopping*.

Hago una serie con 1000 estimadores.

In [ ]:
clf.set_params(n_estimators=1000)
clf.base_estimator.set_params(max_depth=4)
clf.fit(X_train, y_train)
print_results(clf)

Y ahora miro la exactitud en cada paso del proceso.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_val = []
for y_val in clf.staged_predict(X_val):
    accuracy_val.append(accuracy_score(t_val, y_val))

In [ ]:
plt.plot(accuracy_val)
plt.ylim(0.9, 1.0)
plt.axvline(np.argmax(accuracy_val))
plt.axhline(np.max(accuracy_val))

### Gradient Boosting

`sklearn` tiene una clase que implementa *Gradient Boosting* y se llama `GradientBoostingRegressor` y `GradientBoostingClassifier`, y aparece en el paquete `sklearn.ensemble`.

Sin embargo, hay una poderosa librería, que se usa mucho, que se llama `XGBoost`, que implementa Gradient Boosting (Xtreme) de forma eficiente, y que tiene un hermoso paquete para `python`. Entre otras cosas, este paquete imita la API de `sklearn`, de manera que podemos usar la librería casi sin saber nada nuevo. Además, implementa cosas de forma fácil, como el *early stopping*.

In [ ]:
import xgboost

xgb = xgboost.XGBClassifier(verbosity=1, n_estimators=1000)

xgb.fit(X_train, t_train, eval_set=[(X_val, t_val)], early_stopping_rounds=5)
y_val = xgb.predict(X_val)

# 
accuracy_score(t_val, y_val)

## Stacking

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.svm import SVR
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
              ('lasso2', LassoCV(random_state=107)),
              ('svr', SVR(C=1, gamma=1e-6))]

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

reg = StackingRegressor(estimators=estimators, final_estimator=GradientBoostingRegressor(random_state=42))

In [ ]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape, X_test.shape)

In [ ]:
# Y se ajusta como siempre, con la magia de sklearn
reg = reg.fit(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

Y podemos obtener el output de cada uno con el método `transform`.

In [ ]:
reg.transform(X_test)

### Multilayer

In [ ]:
from sklearn.ensemble import RandomForestRegressor

final_layer = StackingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                                            ('gbrt', GradientBoostingRegressor(random_state=42))],
                                final_estimator=RidgeCV())

multi_layer_regressor = StackingRegressor(estimators=[('ridge', RidgeCV()),
                                                      ('lasso', LassoCV(random_state=42)),
                                                      ('svr', SVR(C=1, gamma=1e-6, kernel='rbf'))],
                                          final_estimator=final_layer)

multi_layer_regressor.fit(X_train, y_train)
print('R2 score: {:.2f}'.format(multi_layer_regressor.score(X_test, y_test)))
print(multi_layer_regressor.score(X_test, y_test))